# Evaluate using competition mAP metric
Thanks @theoviel for the starter kernel [here](https://www.kaggle.com/theoviel/competition-metric-map-iou).<br>
This notebook uses rle masks to calculate iou. So, it should be useful for everyone using different models.

## Imports

In [ ]:
import os
import skimage
import numpy as np
import pandas as pd
import skimage.segmentation
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

## Helper Functions

In [ ]:
def rle_decode(mask_rle, shape=[520, 704], color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0::2], s[1::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = color
    return img.reshape(shape)

In [ ]:
def calc_iou_binary(gt_mask, pred_mask):
    "calculate iou between two binary masks."
    try:
        gt_mask = rle_decode(gt_mask).flatten()
        pred_mask = rle_decode(pred_mask).flatten()
        area1 = np.histogram(gt_mask, bins=2)[0]
        area2 = np.histogram(pred_mask, bins=2)[0]
        intersection = np.histogram2d(gt_mask, pred_mask, bins=(2, 2) )[0]
        union = np.expand_dims(area1,1) + np.expand_dims(area2,0) - intersection
        iou = intersection/union
        return iou[1,1]
    except Exception as e:
        print("exception at: calc_iou_binary")
        return 0.0


In [ ]:
def compute_iou(labels, y_pred):
    """
    Computes the IoU for instance labels and predictions.
    Args:
        labels (list): labels.
        y_pred (list): predictions.

    Returns:
        np.array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(labels)
    pred_objects = len(y_pred)
    # initialize iou matrix
    iou_mat = np.zeros((true_objects, pred_objects))
    # compute iou for each true objects and pred objects
    for t in range(true_objects):
        for p in range(pred_objects):
            iou_mat[t,p] = calc_iou_binary(labels[t], y_pred[p])

    return iou_mat

In [ ]:
def precision_at(threshold, iou):
    """
    Computes the precision at a given threshold.

    Args:
        threshold (float): Threshold.
        iou (np array [n_truths x n_preds]): IoU matrix.

    Returns:
        int: Number of true positives,
        int: Number of false positives,
        int: Number of false negatives.
    """
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) >= 1  # Correct objects
    false_negatives = np.sum(matches, axis=1) == 0  # Missed objects
    false_positives = np.sum(matches, axis=0) == 0  # Extra objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn

In [ ]:
def iou_map(truths, preds, verbose=0):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.

    Args:
        truths (list of list of masks): Ground truths.
        preds (list of list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.

    Returns:
        float: mAP.
    """
    ious = [compute_iou(truth, pred) for truth, pred in tqdm(zip(truths, preds))]
    

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tps, fps, fns = 0, 0, 0
        for iou in ious:
            tp, fp, fn = precision_at(t, iou)
            tps += tp
            fps += fp
            fns += fn

        p = tps / (tps + fps + fns)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)

## Load Prediction masks and Truth masks
Load both prediction and truth masks in rle.

### Prediction masks

In [ ]:
from ast import literal_eval
preds_df = pd.read_csv('/kaggle/input/sartvaldataset/val_predictions.csv')
preds_df['masks'] = preds_df['masks'].apply(lambda x: list(literal_eval(x))) # convert to list from str
ids, masks = preds_df['ids'], preds_df['masks']

### Truth masks

In [ ]:
TRAIN_DIR = '/kaggle/input/sartorius-cell-instance-segmentation/train'
val_df = pd.read_csv('/kaggle/input/sartoriuskfold/train_folds.csv')
val_df = val_df[val_df.fold==1]
val_df['file_path'] = val_df['id'].apply(lambda x: os.path.join(TRAIN_DIR, x+'.png'))

In [ ]:
df = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
df = df.groupby('id').agg(list).reset_index()
df = df.merge(val_df, on='id')

df['annotation'] = df['annotation'].apply(
        lambda x: np.unique(x)[0] if len(np.unique(x)) == 1 else np.unique(x)
    )
# create list if list of truth masks
truth_df = df[['id','annotation']]
truth_masks = []
for ann in truth_df['annotation']:
    truth_masks.append(ann) # append list(ann) to truth list

In [ ]:
assert len(ids)==len(masks)==len(truth_masks)

In [ ]:
new_masks = [masks[i] for i in [0, 1, 2, 3, 6, 8]] # take only 6 images for each cell type. (manually selected.)
new_truth_masks = [truth_masks[i] for i in [0, 1, 2, 3, 6, 8]]

In [ ]:
assert len(new_masks)==len(new_truth_masks)

## Compute mAP

In [ ]:
print(f'Validating on {len(new_masks)} images.')
iou_map(new_truth_masks,new_masks,verbose=1)